
# Project 2: Wrangle OpenStreetMap Data

 ## Project : cleaning and sql queries of Annecy open street map data

![https://github.com/MariannePERAUD/Udacity_Project_2_Data_wrangling_with_sql/blob/master/Annecy.jpg](Annecy.jpg)
## Table of Contents
<ul>
<li><a href="#Intro">Introduction</a></li>
<li><a href="#Set-up">Initial Set-up</a></li>   
<li><a href="#assess">Assess data</a></li>
<li><a href="#identify">Identify problems and clean data</a></li>
<li><a href="#transfercsv">Write nodes and ways as csv files</a></li>
<li><a href="#createsql">Create sql databasis</a></li>
<li><a href="#checkdb">Some checks on databasis accuracy and completeness prior to inquiries</a></li>
<li><a href="#querydb">Sql queries</a></li>
<li><a href="#references">Some links to references used</a></li>
</ul>

<a id='Intro'></a>
## Introduction

Area explored is that of Annecy, in the French Alps, a place that I like and where part of my family leaves.
Annecy is near French Alps, not far from Switzerland.



 minlat="45.7996000" minlon="5.9336000" maxlat="45.9323000" maxlon="6.2529000"

I will first check file size and content ; 

Then identify problems and errors and begin data cleaning to make further analysis on a cleaned dataset;  

Transform osm data in csv.files ;  

From csv.files, create a databasisin sql with appropriate structure ;

Analyse databasis with sql



<a id='Set-up'></a>
## Set-up
Import necessary python modules for the analysis and define path to dataset




In [1]:
#!/usr/bin/env python
# coding: utf-8import xml.etree.cElementTree as ET 
import xml.etree.cElementTree as ET 
import codecs ##to write unicode files
import pprint ## to print easier to read dictionnaries
import csv ## to read and write csv file
import os ## to get file size 
import pandas as pd ## I am more familiar with dataframe use than plan dictionnaries
import re ## to search characters in dataset
import sqlite3 ## to use SQL databasis
from collections import defaultdict ## to avoid Keyerrors. Will return default value for missing values. 
                                    ##As advised by Udacity:-)
DATASET = "SmallAnnecy.osm"

pd.set_option('display.width', 2000) ## in order to get nicer display of pandas dataframes on screen
pd.set_option('max_colwidth', -1)

<a id='assess'></a>
   ## Assess data

 ### Check that size of the file is greater than 50Mb#

In [2]:
size=os.path.getsize(DATASET)/1000000
size=round(size)
##Map's size and references
##Source: https://prograide.com/pregunta/6464/obtenir-la-taille-du-fichier-en-python

 ### Count number of different tags in the file#

In [3]:

tags={}
def count_tags(filename):
    for event, elem in ET.iterparse(filename, events=("start",)):
        if elem.tag in tags.keys():
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1
    
    return tags

tags = count_tags(DATASET)

  
    ## see summary at the end of paragraph

 ### Count Number of users as of August 28th 2020 (extraction date)#

In [4]:
def get_user(element):
    return


def process_map(filename):
    """
    Count the user id in the filename.
    """
    users = set()
    for Marianne, element in ET.iterparse(filename):
        try:
            users.add(element.attrib['uid'])
        except KeyError:
            continue

    return users

users = process_map(DATASET)

    ##see summary at the end of paragraph

 ### Statistics summary

In [5]:

print("Size of Annecy.osm file is",size,"Mb, so higher than 50Mb requested")
print("")
print("Number of tags per type")
print(pd.DataFrame([tags]).T)
print("")
print(len(users),"users until August 28th 2020")

Size of Annecy.osm file is 10 Mb, so higher than 50Mb requested

Number of tags per type
              0
osm       1    
note      1    
meta      1    
bounds    1    
node      32582
tag       27879
way       4613 
nd        43133
relation  240  
member    34437

319 users until August 28th 2020


<a id='identify'></a>
   ## Identify and clean problems

Analyses focuses on identifying problems in street names


### Assess various types of street names

French street names are beginning with streetname type,  
Rue Lafayette or Allées Wilson etc...  
In another node, you can find housenumber  
Decoding of Annecy street names has to be a little different from that of US street names.


In [6]:
##FICHAUDIT = "Annecy.osm"

##identifies name of the street in French addresses at the beginning of the address entry, in the form of a string befor

street_type_re = re.compile(r"""
^                                    ## for the beginning of the string
\b                                   ## might be empty, but only at the beginning
\S+                                  ## then followed by any character that is not a space
\.?""",                              
re.IGNORECASE|re.VERBOSE) 



expected = ["Routes","Maison","ZA","ZI","Palais","Parc","Angon","Escaliers","Le","Les","Lieudit","Chemin","Esplanade","Faubourg","Passage","Pont","Port","Rue", "Boulevard","Place","Allée","Avenue","Impasse","Col","Côte","Quai","Rampe","Route","Promenade","Square","Voie"]
##when iterating on the query, we will see that there are many valid possible entries other than rue (=street)

##treet_type_re = re.compile(r'^\b\S+\.?', re.IGNORECASE)


Nested functions in order to audit open street map file

In [7]:

## create street types set(ie strings at the begining of tags before a blanc)
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
           
            street_types[street_type].add(street_name)

## define condition : is in street types
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

## parse all tags for node and way and list all strings corresponding to street types in osm file
def audit(osmfile):
    osm_file = open(DATASET, "r",encoding='utf-8')
    street_types = defaultdict(set)
    ##print (street_types)
    i=0
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        
        if elem.tag == "node" or elem.tag == "way":
           
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])

    return street_types


In [8]:
##run funtion audit on osm file and print "unexpected street type dictionnary"


st_types = audit(DATASET)

pprint.pprint(dict(st_types))


{'fg': {'fg des balmettes'},
 'rue': {'rue Royale',
         'rue Sommeiller',
         'rue de la Gare',
         'rue de la Garr',
         'rue de la gare',
         'rue de la préfecture',
         'rue des Glières',
         'rue du Président Favre',
         'rue du Travail'}}


New mapping dictionary 13 and Georges are not changed at this stage  
                         13 should go in another tag (housenumber) and name of the street type should be added to  
                         street_nametag at a later stage; 
Lower case is replaced and abbreviation fg is replaced by Faubourg.

If users are not paying attention to street names, can we consider dataset as reliable ?
It depends on what we are looking for.
Where I leave, I use open street map for recommended itineraries by bicycle, so street names are not so important for me.
What is more important is what I can find on my way (shops, parks...).

However, let's replace wrong entries in the file that will be imported for queries.


**Replace wrong upercase/lower case**

In [9]:


mapping = { "13": "13",
           "Georges": "Georges",
            "allée.": "Allée",
            "chemin": "Chemin",
            "fg": "Faubourg",
            "route": "Route",
          "rue": "Rue"}

#update name creates a new dictionnary with "before and after"street type entries
def update_name(name):
    street_type= name.split(' ',1)[0]
    street_name= name.split(' ',1)[-1]        

    if street_type in mapping:
        name = mapping[street_type] + ' ' + street_name  

    return name

In [10]:
#run update creates a new dictionnary where street type entries are corrected when necessary
def run_updates(filename):
    
    for st_type, ways in st_types.items():
        for name in ways:
            better_name = update_name(name)
            if better_name != name:
                corrected_names[name] = better_name
    return corrected_names

In [11]:
#corrected names creation for osm file
corrected_names = {}   
corrected_names = run_updates(DATASET)
check2=(pd.DataFrame([corrected_names]).T) 
print("**QUICK CHECK OF MODIFICATIONS PROPOSAL**",check2)

**QUICK CHECK OF MODIFICATIONS PROPOSAL**                                              0
rue des Glières         Rue des Glières       
rue de la Gare          Rue de la Gare        
rue de la Garr          Rue de la Garr        
rue Royale              Rue Royale            
rue du Président Favre  Rue du Président Favre
rue de la préfecture    Rue de la préfecture  
rue de la gare          Rue de la gare        
rue Sommeiller          Rue Sommeiller        
rue du Travail          Rue du Travail        
fg des balmettes        Faubourg des balmettes


<a id='transfercsv'></a>
   ## Write nodes and ways as csv files

File is now ready for import in a csv file and then in SQL databasis.
We will create five csv files preparing five SQL tables creation


In [12]:


NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"



# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

# Prepare modification of dictionnary to import with modified street names
def correct_element(v):
    if v in corrected_names:
        correct_value = corrected_names[v]
    else:
        correct_value = v
    return correct_value

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                   default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        node_attribs['id'] = element.attrib['id']
        node_attribs['user'] = element.attrib['user']
        node_attribs['uid'] = element.attrib['uid']
        node_attribs['version'] = element.attrib['version']
        node_attribs['lat'] = element.attrib['lat']
        node_attribs['lon'] = element.attrib['lon']
        node_attribs['timestamp'] = element.attrib['timestamp']
        node_attribs['changeset'] = element.attrib['changeset']
        
        for node in element:
            tag_dict = {}
            tag_dict['id'] = element.attrib['id']
            if ':' in node.attrib['k']:
                tag_dict['type'] = node.attrib['k'].split(':', 1)[0]
                tag_dict['key'] = node.attrib['k'].split(':', 1)[-1]
                tag_dict['value'] = correct_element(node.attrib['v'])
            else:
                tag_dict['type'] = 'regular'
                tag_dict['key'] = node.attrib['k']
                tag_dict['value'] = correct_element(node.attrib['v'])
            tags.append(tag_dict)
            
    elif element.tag == 'way':
        way_attribs['id'] = element.attrib['id']
        way_attribs['user'] = element.attrib['user']
        way_attribs['uid'] = element.attrib['uid']
        way_attribs['version'] = element.attrib['version']
        way_attribs['timestamp'] = element.attrib['timestamp']
        way_attribs['changeset'] = element.attrib['changeset']
        n = 0
        for node in element:
            if node.tag == 'nd':
                way_dict = {}
                way_dict['id'] = element.attrib['id']
                way_dict['node_id'] = node.attrib['ref']
                way_dict['position'] = n
                n += 1
                way_nodes.append(way_dict)
            if node.tag == 'tag':
                tag_dict = {}
                tag_dict['id'] = element.attrib['id']
                if ':' in node.attrib['k']:
                    tag_dict['type'] = node.attrib['k'].split(':', 1)[0]
                    tag_dict['key'] = node.attrib['k'].split(':', 1)[-1]
                    tag_dict['value'] = correct_element(node.attrib['v'])
                else:
                    tag_dict['type'] = 'regular'
                    tag_dict['key'] = node.attrib['k']
                    tag_dict['value'] = correct_element(node.attrib['v'])
                tags.append(tag_dict)
    
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()



class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, str) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)



In [13]:
# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in):
    """Iteratively process each XML element and write to csv(s)"""

    ## create csv files
    with codecs.open(NODES_PATH, 'w',encoding='utf-8') as nodes_file, \
    codecs.open(NODE_TAGS_PATH, 'w',encoding='utf-8') as nodes_tags_file, \
    codecs.open(WAYS_PATH, 'w',encoding='utf-8') as ways_file, \
    codecs.open(WAY_NODES_PATH, 'w',encoding='utf-8') as way_nodes_file, \
    codecs.open(WAY_TAGS_PATH, 'w',encoding='utf-8') as way_tags_file:

        nodes_writer = csv.DictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = csv.DictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = csv.DictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = csv.DictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = csv.DictWriter(way_tags_file, WAY_TAGS_FIELDS)

    ## create headers in csv files
        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if element.tag == 'node':
                nodes_writer.writerow(el['node'])
                node_tags_writer.writerows(el['node_tags'])
            elif element.tag == 'way':
                ways_writer.writerow((el['way']))
                way_nodes_writer.writerows(el['way_nodes'])
                way_tags_writer.writerows(el['way_tags'])
                    
process_map(DATASET)

<a id='createsql'></a>
   ## Create sql databasis

**Now that .csv are created, create tables that will shelter sql databasis**

In [14]:


# Creating database on disk
sqlite_file = 'Annecy.db'
conn = sqlite3.connect('Annecy.db')
c = conn.cursor()

c.execute('''DROP TABLE IF EXISTS nodes''')
c.execute('''DROP TABLE IF EXISTS nodes_tags''')
c.execute('''DROP TABLE IF EXISTS ways''')
c.execute('''DROP TABLE IF EXISTS ways_tags''')
c.execute('''DROP TABLE IF EXISTS ways_nodes''')
conn.commit()



**Create sql tables**

In [15]:
QUERY_NODES = """


CREATE TABLE nodes (
    id INTEGER NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);
"""

QUERY_NODES_TAGS = """
CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id)
);
"""

QUERY_WAYS = """
CREATE TABLE ways (
    id INTEGER NOT NULL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);
"""

QUERY_WAYS_TAGS = """
CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id)
);
"""

QUERY_WAYS_NODES = """
CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id)
);
"""



c.execute(QUERY_NODES)
c.execute(QUERY_NODES_TAGS)
c.execute(QUERY_WAYS)
c.execute(QUERY_WAYS_TAGS)
c.execute(QUERY_WAYS_NODES)

conn.commit()

In [16]:
with open('nodes.csv','rt',encoding='utf8') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db1 = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]
    
with open('nodes_tags.csv','rt',encoding='utf8') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db2 = [(i['id'], i['key'], i['value'], i['type']) for i in dr]
    
with open('ways.csv','rt',encoding='utf8') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db3 = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]
    
with open('ways_tags.csv','rt',encoding='utf8') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db4 = [(i['id'], i['key'], i['value'], i['type']) for i in dr]
    
with open('ways_nodes.csv','rt',encoding='utf8') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db5 = [(i['id'], i['node_id'], i['position']) for i in dr]

In [17]:
c.executemany("INSERT INTO nodes(id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db1)
c.executemany("INSERT INTO nodes_tags(id, key, value, type) VALUES (?, ?, ?, ?);", to_db2)
c.executemany("INSERT INTO ways(id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db3)
c.executemany("INSERT INTO ways_tags(id, key, value, type) VALUES (?, ?, ?, ?);", to_db4)
c.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?, ?, ?);", to_db5)
conn.commit()


<a id='checkdb'></a>
   ## Some checks on databasis accuracy and completeness prior to inquiries

Several preliminary checks on SQL databasis
- Check that databasis is complete (same number of nodes and ways as computed before)
- Check that some wrong street names have been corrected

In [18]:
c.execute('SELECT COUNT(*) FROM nodes')
all_rows = c.fetchall()
print(all_rows)

c.execute('SELECT COUNT(*) FROM ways')
all_rows = c.fetchall()
print(all_rows)

[(32582,)]
[(4613,)]


In [19]:
CHECK_CHANGES2= """
SELECT value 
FROM ways_tags
WHERE value ='Chemin de Bellevue'
OR value ='Rue Cassiopée'
OR value ='rue Cassiopée'
OR value ='Chemin de Bellevue'
limit 10;
"""
c.execute(CHECK_CHANGES2)
all_rows=c.fetchall()
print(all_rows)



[]


In [20]:
CHECK_CHANGES3= """
SELECT value 
FROM nodes_tags
WHERE value ='Chemin de Bellevue'
OR value ='Rue Cassiopée'
OR value ='rue Cassiopée'

;
"""
c.execute(CHECK_CHANGES3)
all_rows=c.fetchall()
print(all_rows)



[]


<a id='querydb'></a>
   ## SQL queries

_Enquiries about content of the database in SQL, at least..._

**Count number of most active users**


In [21]:
# Query to show the nicknames *user* and contributions of the top 15 contributors
QUERY = '''
SELECT DISTINCT nodes.user as USER, COUNT(*) as contribution_number
FROM nodes
GROUP BY nodes.user
ORDER BY COUNT(*) DESC
LIMIT 15;
'''
c.execute(QUERY)
all_rows = c.fetchall()
contributors=pd.DataFrame([all_rows]).T
contributors.columns=(["users, number of contributions"])
print(contributors)


   users, number of contributions
0   (botdidier2020, 6778)        
1   (Emmanuel Pacaud, 6477)      
2   (Virgile1994, 5640)          
3   (InfosReseaux, 2615)         
4   (chimel38, 2123)             
5   (Zedh, 1653)                 
6   (Marc Mongenet, 1197)        
7   (didier2020, 563)            
8   (Musculus, 393)              
9   (pyrog, 370)                 
10  (dom74, 328)                 
11  (Fabien98, 294)              
12  (Mickael42, 269)             
13  (Verdy_p, 261)               
14  (cquest, 228)                


**What are main shops keys and number for each key, list first 40**

In [22]:
LIST_NTSHOPS="""
select value,count(value)
from nodes_tags
WHERE key ='shop'
group by value
order by count(value) desc

;
"""
c.execute(LIST_NTSHOPS)
all_rows=c.fetchall()
z=pd.DataFrame([all_rows]).T
print(z.iloc[0:20,])


                            0
0   (clothes, 82)            
1   (hairdresser, 19)        
2   (bakery, 18)             
3   (interior_decoration, 15)
4   (shoes, 13)              
5   (optician, 13)           
6   (books, 10)              
7   (beauty, 10)             
8   (furniture, 8)           
9   (convenience, 8)         
10  (art, 8)                 
11  (jewelry, 7)             
12  (supermarket, 6)         
13  (sports, 6)              
14  (mobile_phone, 6)        
15  (estate_agent, 4)        
16  (deli, 4)                
17  (butcher, 4)             
18  (bicycle, 4)             
19  (bag, 4)                 


In [23]:
print(z.iloc[20:40,])


                          0
20  (weapons, 3)           
21  (travel_agency, 3)     
22  (tobacco, 3)           
23  (tattoo, 3)            
24  (musical_instrument, 3)
25  (laundry, 3)           
26  (kitchen, 3)           
27  (gift, 3)              
28  (frame, 3)             
29  (confectionery, 3)     
30  (beverages, 3)         
31  (alcohol, 3)           
32  (toys, 2)              
33  (ticket, 2)            
34  (sewing, 2)            
35  (pastry, 2)            
36  (outdoor, 2)           
37  (music, 2)             
38  (houseware, 2)         
39  (household_linen, 2)   


**What are the main amenities, list first 50 and count for each key**

In [24]:
LIST_NTAMEN="""
select value,count(value)
from nodes_tags
WHERE key ='amenity'
group by value
order by count(value) desc

;
"""
c.execute(LIST_NTAMEN)
all_rows=c.fetchall()
z=pd.DataFrame([all_rows]).T
print(z.iloc[0:50,])


                          0
0   (restaurant, 118)      
1   (bicycle_parking, 110) 
2   (waste_basket, 87)     
3   (bench, 74)            
4   (shelter, 29)          
5   (fast_food, 29)        
6   (cafe, 27)             
7   (drinking_water, 26)   
8   (vending_machine, 23)  
9   (recycling, 21)        
10  (parking_entrance, 17) 
11  (post_box, 14)         
12  (bar, 14)              
13  (pharmacy, 13)         
14  (toilets, 12)          
15  (bank, 12)             
16  (atm, 11)              
17  (charging_station, 10) 
18  (ferry_terminal, 9)    
19  (pub, 8)               
20  (school, 7)            
21  (fountain, 6)          
22  (car_rental, 6)        
23  (ice_cream, 5)         
24  (bicycle_rental, 4)    
25  (theatre, 3)           
26  (social_facility, 3)   
27  (place_of_worship, 3)  
28  (parking_space, 3)     
29  (parking, 3)           
30  (nightclub, 3)         
31  (library, 3)           
32  (kindergarten, 3)      
33  (driving_school, 3)    
34  (taxi, 2)       

**What are main "historic" keys ?**

In [25]:
LIST_HISTORIC="""
select value,count(value)
from nodes_tags
WHERE key ='historic'
group by value
order by count(value) desc

;
"""

c.execute(LIST_HISTORIC)
all_rows=c.fetchall()
z=pd.DataFrame([all_rows]).T
print(z)

                0
0  (memorial, 13)
1  (yes, 1)      
2  (ruins, 1)    


**There are 5 archeological sites, what are other informations (nodes_tags) about those archeological sites**

In [26]:
ARCH_HISTORIC="""
select nodes_tags.id, nodes_tags.key, nodes_tags.value from nodes_tags,
((select nodes.id as sitesarcheo
from  nodes_tags left join nodes
ON nodes.id=nodes_tags.id
AND nodes_tags.key ='historic'
AND nodes_tags.value ='archaeological_site') as sousprog)

WHERE nodes_tags.id = sitesarcheo
order by nodes_tags.id
;
"""

c.execute(ARCH_HISTORIC)
all_rows=c.fetchall()
z=pd.DataFrame([all_rows]).T
print(z)

Empty DataFrame
Columns: [0]
Index: []


<a id='references'></a>
   ## Some links to references used


- Map feature in openstreetmap https://wiki.openstreetmap.org/wiki/Map_Features (list of nodes and tags to use)

Solutions to some python coding problems
- VERBOSE and Re Module https://www.geeksforgeeks.org/verbose-in-python-regex/
- csv write and headers https://discuss.codecademy.com/t/how-does-writeheader-know-the-fields-to-use/463772  
- how to get nice display of pandas dataframe (correct width) https://stackoverflow.com/questions/11707586/how-do-i-expand-the-output-display-to-see-more-columns-of-a-pandas-dataframe